In [9]:
import requests as rq
import re
from bs4 import BeautifulSoup as BS

In [10]:
ADDRESSE = "https://www.vinatis.com/61101-les-darons-2022-by-jeff-carrel"
ADDRESSE2 =  "https://www.vinatis.com/42172-pinot-noir-harmonie-bollenberg-2015-domaine-valentin-zusslin"

In [11]:
def _scrap_soupe(adresse:str) -> BS:
    requete = rq.get(url=adresse, timeout=10)
    soupe = BS(requete.text)
    return soupe

In [12]:
def _scrap_name(soupe:BS) -> str:
    name = soupe.find_all(name="span", attrs={"itemprop": "name"})[0].text
    return name

In [13]:
def _scrap_capacity(soupe:BS) -> str:
    capacity = soupe.find_all(name="span", attrs={"inline-block"})[0].text
    return capacity

In [14]:
def _scrap_price(soupe:BS) -> str:
    price = soupe.find_all(name="span", attrs={"id" : "our_price_display"})[0].text
    
    try:
        price_discount_1 = soupe.find_all(name="span", attrs={"id" : "quantity_discount_pretaxe"})[0].text
    except:
        price_discount_1 = None
        
    try:
        price_discount_2 = soupe.find_all(name="span", attrs={"id" : "quantity_discount_pretaxe"})[1].text
    except:
        price_discount_2 = None
    return price, price_discount_1, price_discount_2

In [15]:
def _scrap_characteristic(soupe:BS)->str:
    characteristic = soupe.find_all(name="span", attrs={"class": "no-padding-horizontal"})[0].text
    return characteristic

In [16]:
def _scrap_note(soupe:BS)-> str:
    note = soupe.find_all(name="div", attrs={"class": "col-xs-12 padding-bottom-10"})[0].text
    return note

In [17]:
def _scrap_keyword(soupe:BS)-> list:
    keyword = []
    class_k = "margin-right margin-bottom bg-gray-dark taille-md padding-horizontal-30 padding-vertical-5 rounded-corner-3 label"
    result = soupe.find_all(name = "span", attrs={"class": class_k})
    for res in result:
        keyword.append(res.text)
    return keyword


In [18]:
def _scrap_informations(soupe:BS) -> str:
    class_ = "table-cell-css vertical-align-top padding-vertical-5 taille-xs color-gray-darker text-bold"
    vintage = soupe.find_all(name="div", attrs={"class": class_})[0].text
    grap_variety = soupe.find_all(name="div", attrs={"class": class_})[1].text
    
    taste = soupe.find_all(name="div", attrs={"class": class_})[2].text
    taste_group = soupe.find_all(name="div", attrs={"class": class_})[3].text
    eyes = soupe.find_all(name="div", attrs={"class": class_})[4].text
    nose = soupe.find_all(name="div", attrs={"class": class_})[5].text
    mouth = soupe.find_all(name="div", attrs={"class": class_})[6].text
    
    temperature = soupe.find_all(name="div", attrs={"class": class_})[7].text
    indication = soupe.find_all(name="div", attrs={"class": class_})[8].text
    conservation = soupe.find_all(name="div", attrs={"class": class_})[9].text
    max_date = soupe.find_all(name="div", attrs={"class": class_})[10].text
    
    accompaniement = soupe.find_all(name="div", attrs={"class": class_})[11].text
    
    return vintage, grap_variety, taste, taste_group, eyes, nose, mouth, temperature, indication, conservation, max_date, accompaniement

In [19]:
def _scrap_img(soupe:BS) -> str:
    picture = soupe.find(name = "img", attrs={"class":"img-full-width img-max-450 center-block"}).get('src')
    return picture

In [20]:
from dataclasses import dataclass
from serde import serialize
from serde.json import to_json

@serialize
@dataclass
class Vin:
    """This dataclass represents all characteristics associated with a Wine."""

    name: str
    capacity: str
    prices : str
    characteristic : str
    note: str
    keyword : str
    informations : str
    picture : str


In [21]:
def _scraping(adresse):
    soupe = _scrap_soupe(adresse)
    
    results = Vin(
        _scrap_name(soupe),
        _scrap_capacity(soupe),
        _scrap_price(soupe),
        _scrap_characteristic(soupe),
        _scrap_note(soupe),
        _scrap_keyword(soupe),
        _scrap_informations(soupe),
        _scrap_img(soupe)
        )
    
    return results

In [22]:
def json_brut(adresse):
    vins = _scraping(adresse)
    vins_json = to_json(vins)
    
    file_path = "data/vins.json"

    with open(file_path, "w",  encoding='utf-8') as json_file:
        json_file.write(vins_json)

In [23]:
json_brut(ADDRESSE2)

In [24]:
import src.modules.scraping as sc

In [25]:
sc.scraping("https://www.vinatis.com/60060-monsieur-n-rouge-grenache-syrah-maison-francois-xavier-nicolas")

c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\src\modules\scraping.py:9: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 9 of the file c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\src\modules\scraping.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soupe = BS(requete.text)


Vin(name='MONSIEUR N ROUGE - GRENACHE, SYRAH - MAISON FRANÇOIS-XAVIER NICOLAS', capacity=' 0,75 L ', prices=('  7,00 €    ', ' 6,30 € ', None), characteristic='Vin Rouge / Vin de France / 13,5 % vol', note='', keyword=['Gourmandise', 'Grenache', 'Syrah'], informations=(' non millésimé', ' Grenache, Syrah', ' Rouge Charnu et fruité', ' Fruité', ' Robe pourpre aux reflets vifs', " Nez intense de fruits rouges (framboises) et des notes d'épices douces", ' Le grenache apporte une bouche très riche et équilibrée, toute en rondeur et offre des arômes de coulis de petits fruits frais. la Syrah, elle, apporte le côté fruits sauvages et floral (cassis et violette)', ' 17-18°C', ' En bouteille', ' A boire et à garder', ' 2028', ' Apéritif, Charcuterie, Viande rouge, Viande blanche, Volaille, Barbecue, Cuisine du monde'), picture='https://www.vinatis.com/78203-detail_default/monsieur-n-rouge-grenache-syrah-maison-francois-xavier-nicolas.png')

In [159]:
def _scrap_informations(soupe:BS) -> str:
    class_n = "table-cell-css vertical-align-top padding-vertical-5 nowrap padding-right-10 taille-xs color-gray-darker"
    class_value = "table-cell-css vertical-align-top padding-vertical-5 taille-xs color-gray-darker text-bold"
    
    details = [' \xa0\xa0Millésime',' \xa0\xa0Cépage',' \xa0\xa0Goûts',
           ' \xa0\xa0Par Goûts', " \xa0\xa0A l'oeil", ' \xa0\xa0Au nez',
           ' \xa0\xa0En bouche',' \xa0\xa0Température de service',' \xa0\xa0Service', 
           ' \xa0\xa0Conservation', " \xa0\xa0Jusqu'à", ' \xa0\xa0Accords mets-vin',
           ' \xa0\xa0Accords recommandés']
   
    dictionnaire_vin = {
        ' \xa0\xa0Millésime': [],
        ' \xa0\xa0Cépage': [],
        ' \xa0\xa0Goûts': [],
        ' \xa0\xa0Par Goûts': [],
        " \xa0\xa0A l'oeil": [],
        ' \xa0\xa0Au nez': [],
        ' \xa0\xa0En bouche': [],
        ' \xa0\xa0Température de service': [],
        ' \xa0\xa0Service': [],
        ' \xa0\xa0Conservation': [],
        " \xa0\xa0Jusqu'à": [],
        ' \xa0\xa0Accords mets-vin': [],
        ' \xa0\xa0Accords recommandés': []
    }
   
    place_element = soupe.find_all(name="div", attrs={"class": class_n})
    value_element = soupe.find_all(name="div", attrs={"class": class_value})
    details_page = [place_element[i].text for i in range(len(place_element))]
    value_page = [value_element[i].text for i in range(len(value_element))]
    
    j = 0
    for i, detail in zip(range(13),details):
        try:
            details_page.index(details[i])
            dictionnaire_vin[detail] = value_page[j]
            j = j+1
        except:
            dictionnaire_vin[detail] = None
        
    
    return dictionnaire_vin

In [160]:
soupe = _scrap_soupe("https://www.vinatis.com/52605-vacqueyras-vieilles-vignes-2020-domaine-sandilaure")
_scrap_informations(soupe)

{' \xa0\xa0Millésime': ' 2020',
 ' \xa0\xa0Cépage': None,
 ' \xa0\xa0Goûts': ' Rouge Puissant et généreux',
 ' \xa0\xa0Par Goûts': ' Puissant',
 " \xa0\xa0A l'oeil": ' Belle robe grenat.',
 ' \xa0\xa0Au nez': ' Complexe et très gourmand sur les fruits mûrs et compotés complété par des nuances épicées.',
 ' \xa0\xa0En bouche': ' Une bouche intense et structurée, aux saveurs de fruits (fruits rouges, pruneaux, figue), et jolies notes épicées en finale.',
 ' \xa0\xa0Température de service': ' 16-18°C',
 ' \xa0\xa0Service': ' Ouvrir 1h avant le service',
 ' \xa0\xa0Conservation': ' A boire et à garder',
 " \xa0\xa0Jusqu'à": ' 2032',
 ' \xa0\xa0Accords mets-vin': ' Charcuterie, Viande rouge, Viande blanche, Barbecue, Cuisine du monde, Fromage',
 ' \xa0\xa0Accords recommandés': ' Grillades, entrecôte, sauté de canard, poivrons farcis, risotto au chorizo'}

In [129]:
class_n = "table-cell-css vertical-align-top padding-vertical-5 nowrap padding-right-10 taille-xs color-gray-darker"
class_value = "table-cell-css vertical-align-top padding-vertical-5 taille-xs color-gray-darker text-bold"

In [130]:
details = [' \xa0\xa0Millésime',' \xa0\xa0Cépage',' \xa0\xa0Goûts',
           ' \xa0\xa0Par Goûts', " \xa0\xa0A l'oeil", ' \xa0\xa0Au nez',
           ' \xa0\xa0En bouche',' \xa0\xa0Température de service',' \xa0\xa0Service', 
           ' \xa0\xa0Conservation', " \xa0\xa0Jusqu'à", ' \xa0\xa0Accords mets-vin',
           ' \xa0\xa0Accords recommandés']

In [131]:
place_element = soupe.find_all(name="div", attrs={"class": class_n})
value_element = soupe.find_all(name="div", attrs={"class": class_value})
details_page = [place_element[i].text for i in range(len(place_element))]
value_page = [value_element[i].text for i in range(len(value_element))]

In [122]:
dictionnaire_vin = {
    ' \xa0\xa0Millésime': [],
    ' \xa0\xa0Cépage': [],
    ' \xa0\xa0Goûts': [],
    ' \xa0\xa0Par Goûts': [],
    " \xa0\xa0A l'oeil": [],
    ' \xa0\xa0Au nez': [],
    ' \xa0\xa0En bouche': [],
    ' \xa0\xa0Température de service': [],
    ' \xa0\xa0Service': [],
    ' \xa0\xa0Conservation': [],
    " \xa0\xa0Jusqu'à": [],
    ' \xa0\xa0Accords mets-vin': [],
    ' \xa0\xa0Accords recommandés': []
}

In [154]:
j = 0
for i, detail in zip(range(13),details):
    try:
        details_page.index(details[i])
        dictionnaire_vin[detail] = value_page[j]
        j = j+1
    except:
        dictionnaire_vin[detail] = None
    

In [156]:
dictionnaire_vin

{' \xa0\xa0Millésime': ' 2020',
 ' \xa0\xa0Cépage': None,
 ' \xa0\xa0Goûts': ' Rouge Puissant et généreux',
 ' \xa0\xa0Par Goûts': ' Puissant',
 " \xa0\xa0A l'oeil": ' Belle robe grenat.',
 ' \xa0\xa0Au nez': ' Complexe et très gourmand sur les fruits mûrs et compotés complété par des nuances épicées.',
 ' \xa0\xa0En bouche': ' Une bouche intense et structurée, aux saveurs de fruits (fruits rouges, pruneaux, figue), et jolies notes épicées en finale.',
 ' \xa0\xa0Température de service': ' 16-18°C',
 ' \xa0\xa0Service': ' Ouvrir 1h avant le service',
 ' \xa0\xa0Conservation': ' A boire et à garder',
 " \xa0\xa0Jusqu'à": ' 2032',
 ' \xa0\xa0Accords mets-vin': ' Charcuterie, Viande rouge, Viande blanche, Barbecue, Cuisine du monde, Fromage',
 ' \xa0\xa0Accords recommandés': ' Grillades, entrecôte, sauté de canard, poivrons farcis, risotto au chorizo'}

In [1]:
import src.modules.scraping as sc

In [2]:
sc.scraping("https://www.vinatis.com/52605-vacqueyras-vieilles-vignes-2020-domaine-sandilaure")

c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\src\modules\scraping.py:9: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 9 of the file c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\src\modules\scraping.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soupe = BS(requete.text)


Vin(name='VACQUEYRAS VIEILLES VIGNES 2020 - DOMAINE SANDILAURE', capacity=' 0,75 L ', prices=('  15,50 €    ', ' 11,90 € ', None), characteristic='Vin Rouge / Rhône / Vacqueyras AOC / 14 % vol', note=' 16.9/20  noté par 13 clients ', keyword=['Epicé', 'Charpenté', 'Maison incontournable'], informations={' \xa0\xa0Millésime': ' 2020', ' \xa0\xa0Cépage': None, ' \xa0\xa0Goûts': ' Rouge Puissant et généreux', ' \xa0\xa0Par Goûts': ' Puissant', " \xa0\xa0A l'oeil": ' Belle robe grenat.', ' \xa0\xa0Au nez': ' Complexe et très gourmand sur les fruits mûrs et compotés complété par des nuances épicées.', ' \xa0\xa0En bouche': ' Une bouche intense et structurée, aux saveurs de fruits (fruits rouges, pruneaux, figue), et jolies notes épicées en finale.', ' \xa0\xa0Température de service': ' 16-18°C', ' \xa0\xa0Service': ' Ouvrir 1h avant le service', ' \xa0\xa0Conservation': ' A boire et à garder', " \xa0\xa0Jusqu'à": ' 2032', ' \xa0\xa0Accords mets-vin': ' Charcuterie, Viande rouge, Viande blan

In [3]:
sc.json_brut(sc.scraping("https://www.vinatis.com/52605-vacqueyras-vieilles-vignes-2020-domaine-sandilaure"))

c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\src\modules\scraping.py:9: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 9 of the file c:\Users\guill\OneDrive - Université de Tours\Bureau\M2\ML\Wine-Scraping\src\modules\scraping.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soupe = BS(requete.text)
